In [4]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import dask.array as da
import xarray as xr
import os


from xmitgcm import open_mdsdataset
import xmitgcm
import ecco_v4_py as ecco


from netCDF4 import Dataset

import seawater

from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import derive_potential_density_values_TEST
from analysis_package import ecco_masks

from importlib import reload

# reload modules for prototyping...
ecco_masks = reload(ecco_masks)
plotting_functions = reload(plotting_functions)
open_datasets = reload(open_datasets)
derive_potential_density_values_TEST = reload(derive_potential_density_values_TEST)


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
############################################################################################################
#############################################     LOAD DATA    #############################################
############################################################################################################

data_dir = "./nctiles_monthly/"
time_slice = np.arange(0,288)

oceFWflx = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,"oceFWflx",time_slice,rename_indices=True)
oceQnet = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,"oceQnet",time_slice,rename_indices=True)
P_INSITU = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,"P_INSITU",time_slice,rename_indices=False).isel(k=0)
T_INSITU = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,"T_INSITU",time_slice,rename_indices=False).isel(k=0)
SALT = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,"SALT",time_slice,rename_indices=True).isel(k=0)

Loaded oceFWflx over time slice  

Loaded oceQnet over time slice  

Loaded P_INSITU over time slice  

Loaded T_INSITU over time slice  

Loaded SALT over time slice  



In [6]:
print(oceFWflx)
print("\n")
print(oceQnet)
print("\n")
print(P_INSITU)
print("\n")
print(T_INSITU)
print("\n")
print(SALT)

<xarray.Dataset>
Dimensions:   (i: 90, j: 90, tile: 13, time: 288)
Coordinates:
  * time      (time) float64 1.0 2.0 3.0 4.0 5.0 ... 285.0 286.0 287.0 288.0
  * j         (j) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 86.0 87.0 88.0 89.0 90.0
  * i         (i) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 86.0 87.0 88.0 89.0 90.0
    tim       (time) datetime64[ns] 1992-01-16 1992-02-16 ... 2015-12-16
    lon       (tile, j, i) float64 -111.6 -111.3 -110.9 ... -99.42 -105.6 -111.9
    lat       (tile, j, i) float64 -88.24 -88.38 -88.52 ... -88.03 -88.08 -88.1
Dimensions without coordinates: tile
Data variables:
    oceFWflx  (tile, time, j, i) float64 nan nan nan nan nan ... nan nan nan nan
    timstep   (tile, time) float64 732.0 1.428e+03 ... 2.096e+05 2.104e+05
    land      (tile, j, i) float64 nan nan nan nan nan ... nan nan nan nan nan
    area      (tile, j, i) float64 3.623e+08 3.633e+08 ... 3.685e+08 3.611e+08
Attributes:
    description:    oceFWflx -- ECCO v4 ocean state estimate, release 3 

In [9]:
# The python seawater package uses salinity in psu, temperature in C (ITS-90), and pressure in db.
# Make sure to verify units before processing
surface_alpha = seawater.eos80.alpha(SALT.SALT, T_INSITU.TEMP_INSITU, P_INSITU.P_INSITU)
print("done surface alpha")
surface_beta = seawater.eos80.beta(SALT.SALT, T_INSITU.TEMP_INSITU, P_INSITU.P_INSITU)
print("done surface beta")
surface_heat_capacity = seawater.eos80.cp(SALT.SALT, T_INSITU.TEMP_INSITU, P_INSITU.P_INSITU)
print("done surface heat capacity")

done surface alpha
done surface beta
done surface heat capacity


In [16]:
print(surface_alpha.shape)
print(oceQnet.oceQnet.shape)

(13, 288, 90, 90)
(13, 288, 90, 90)


In [18]:
surface_alpha_ds = (oceQnet.oceQnet.copy(deep=True)*0 + surface_alpha)#.rename({"oceQnet":"alpha"})
surface_beta_ds = (oceQnet.oceQnet.copy(deep=True)*0 + surface_beta)#.rename({"oceQnet":"beta"})
surface_cp_ds = (oceQnet.oceQnet.copy(deep=True)*0 + surface_heat_capacity)#.rename({"oceQnet":"heat_capacity"})

In [20]:

surface_alpha_ds.to_netcdf("surface_alpha.nc")
surface_beta_ds.to_netcdf("surface_beta.nc")
surface_cp_ds.to_netcdf("surface_heat_capacity.nc")

